In [0]:
# !pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%reload_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
# cd drive/My\ Drive/Colab \Notebooks/dw_matrix/matrix_three

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, 
            y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs=params_fit.get('epochs', 5),
            verbose=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (X_train, y_train)),
            callbacks=[tensorboard_callback]
            )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(0.3),
                     
                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(0.3),
                     
                    Dense(1024, activation='relu'),
                    Dropout(0.3),

                    Dense(num_classes, activation='softmax')
  ])

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 48ms/step - loss: 2.9136 - accuracy: 0.2236 - val_loss: 1.3701 - val_accuracy: 0.5397
Epoch 2/5
272/272 [==============================] - 12s 46ms/step - loss: 1.1173 - accuracy: 0.6328 - val_loss: 0.3443 - val_accuracy: 0.8834
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.4739 - accuracy: 0.8503 - val_loss: 0.1201 - val_accuracy: 0.9670
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2717 - accuracy: 0.9177 - val_loss: 0.0501 - val_accuracy: 0.9852
Epoch 5/5
272/272 [==============================] - 12s 46ms/step - loss: 0.2092 - accuracy: 0.9396 - val_loss: 0.0401 - val_accuracy: 0.9893


0.9653061224489796

In [11]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.1761 - accuracy: 0.9653


0.9653061032295227

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
                    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_one']),
                     
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_two']),
                     
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
                    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
                    MaxPool2D(),
                    Dropout(params['dropout_cnn_block_three']),
                     
                    Flatten(),

                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_one']),
                     
                    Dense(1024, activation='relu'),
                    Dropout(params['dropout_dense_block_two']),

                    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):  
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [44]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two',  0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three',  0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one',  0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',  0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4452015461330191, 'dropout_cnn_block_three': 0.30715877113741125, 'dropout_cnn_block_two': 0.3786336227180305, 'dropout_dense_block_one': 0.396487804532351, 'dropout_dense_block_two': 0.6692671445173384}
accuracy=0.9181405901908875
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.3203644204722046, 'dropout_cnn_block_three': 0.3257419501236324, 'dropout_cnn_block_two': 0.39865339096917485, 'dropout_dense_block_one': 0.5690195982251733, 'dropout_dense_block_two': 0.6575853448713052}
accuracy=0.9653061032295227
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.35770396920548275, 'dropout_cnn_block_three': 0.4874186506802818, 'dropout_cnn_block_two': 0.373511891827217, 'dropout_dense_block_one': 0.5953180969749221, 'dropout_dense_block_two': 0.4884723261328455}
accuracy=0.9580498933792114
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.37348249263931965, 'dropout_cnn_block_three': 0.3009620892425484, 'dropout_cnn_block_two': 0.43017920845319